In [1353]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [1354]:
# derivative explanations by Andrej Karpathy
h = 0.0001

# inputs 
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
# Quanto uma mudança nessa variável isolada afeta a curva da função?
b += h
d2 = a*b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1)/h)

d1 4.0
d2 4.0002
slope 2.0000000000042206


In [1355]:
# Precisamos de estruturas de dados que possam funcionar em uma rede neural
# Vamos reconstruir a função básica do bloco anterior com uma simples classe 
# que lida com os valores

class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data  = data
        self.grad  = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op   = _op
        self.label = label
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data+other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
            
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data*other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "apenas suporta potencias de tipo int/float"
        out = Value(self.data**other, (self, ), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self + (-other)
    
    def __rsub__(self, other):
        return other + (-self)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
            

In [1356]:
from graphviz import Digraph

def trace(root):
    # constrói um set com todos os nós e vértices de um grafo
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR - left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # para cada valor no grafo, cria um nó ('record') retangular para ele
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # se esse valor é o resultado de alguma operação, cria um nó op para ele
            dot.node(name = uid + n._op, label = n._op)
            # e conecta o nó a ele
            dot.edge(uid + n._op, uid)
        
    for n1, n2 in edges:
        # conecta n1 ao nó op de n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

In [1357]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias of the neuron
b  = Value(6.7, label='b')

# x1w1 +x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'

# ----
e = (2*n).exp()
o = (e - 1) / (e + 1)
# ----
o.label = 'o'
o.backward()

# draw_dot(n)


In [1358]:
import torch

In [1359]:
x1 = torch.Tensor([2.0]).double()                   ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                   ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()                  ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                   ; w2.requires_grad = True
b  = torch.Tensor([6.8813735870195432]).double()    ; w2.requires_grad = True
n  = x1*w1 + x2*w2 + b
o  = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [1360]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [1361]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

[Value(data=-0.40393108075953676)]

In [1362]:
xs = [ # inputs
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0] # alvos

In [1363]:
for k in range(20):
    # forward pass
    ypred = [n(x)[0] for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    
    # backward pass
    for p in n.parameters():
        p.grad = 0
    loss.backward()
    
    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    
    print(k, loss.data)

0 4.6099452555460445
1 3.3856046774368953
2 3.090614450306447
3 2.6931925253032616
4 2.1830561514018445
5 1.618421878846169
6 1.1136757565128372
7 0.7448516833454275
8 0.5071999496497854
9 0.3609448865713968
10 0.2696251806678488
11 0.2101949119793406
12 0.1696762760861671
13 0.140841534705271
14 0.11954906795047088
15 0.10332792920059915
16 0.09064177432975021
17 0.0804979948376411
18 0.07223284244203985
19 0.06538877116148008


In [1352]:
ypred

[Value(data=0.9962278216337767),
 Value(data=-0.9963513814847642),
 Value(data=-0.9967985876784696),
 Value(data=0.9964559424651581)]